# Go2-W (Wheel-Leg Hybrid) Robot Simulation

This notebook is specifically for the **Unitree Go2-W** - the wheel-legged hybrid version!

The Go2-W has:
- 16 degrees of freedom (vs 12 on standard Go2)
- Wheel feet for smooth surfaces
- Leg locomotion for rough terrain
- 4D LiDAR (360°×90° coverage)
- 100 TOPS computing (Nvidia Jetson Orin NX)

---

## Method 1: Official Unitree MuJoCo Simulator (Recommended)

This is the official simulator that **supports Go2-W directly**!

### Step 1: Install Dependencies

In [ ]:
# Install Python dependencies
!pip install mujoco numpy matplotlib pygame

In [ ]:
# Clone the official Unitree MuJoCo simulator
!git clone https://github.com/unitreerobotics/unitree_mujoco.git

In [ ]:
# Install Unitree SDK2 Python
!git clone https://github.com/unitreerobotics/unitree_sdk2_python.git
!cd unitree_sdk2_python && pip install -e .

### Step 2: Check Available Robot Models

Let's see what robots are available in the simulator:

In [ ]:
import os

# List available robots in unitree_mujoco
robots_dir = 'unitree_mujoco/unitree_robots'

if os.path.exists(robots_dir):
    robots = os.listdir(robots_dir)
    print("Available robot models:")
    for robot in sorted(robots):
        robot_path = os.path.join(robots_dir, robot)
        if os.path.isdir(robot_path):
            # Check for scene.xml
            scene_file = os.path.join(robot_path, 'scene.xml')
            has_scene = '✓' if os.path.exists(scene_file) else '✗'
            print(f"  {has_scene} {robot}")
else:
    print("Please run the git clone cell first!")

### Step 3: Configure for Go2-W

The simulator config needs to be set to `go2w` (not `go2`):

In [ ]:
# Create/update the Python config for Go2-W
config_content = '''
# Configuration for Go2-W (Wheel-Leg Hybrid)

# IMPORTANT: Use "go2w" for the wheel-leg version!
ROBOT = "go2w"  # <-- This is the key difference from standard Go2

# Robot simulation scene file
ROBOT_SCENE = "../unitree_robots/" + ROBOT + "/scene.xml"

# DDS domain id (use 1 for simulation, 0 for real robot)
DOMAIN_ID = 1

# Network interface ("lo" for simulation)
INTERFACE = "lo"

# Print scene information
PRINT_SCENE_INFORMATION = True

# Joystick settings (set to 0 if no gamepad)
USE_JOYSTICK = 0
JOYSTICK_TYPE = "xbox"
JOYSTICK_DEVICE = 0

# Simulation settings
ENABLE_ELASTIC_BAND = False
SIMULATE_DT = 0.003
VIEWER_DT = 0.02
'''

# Write config file
config_path = 'unitree_mujoco/simulate_python/config.py'
if os.path.exists('unitree_mujoco/simulate_python'):
    with open(config_path, 'w') as f:
        f.write(config_content)
    print(f"✓ Config updated for Go2-W at {config_path}")
else:
    print("Please clone unitree_mujoco first!")

### Step 4: Load and View Go2-W Model

Now let's load the actual Go2-W model:

In [ ]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt

# Path to Go2-W model
go2w_scene_path = 'unitree_mujoco/unitree_robots/go2w/scene.xml'

# Check if Go2-W model exists
if os.path.exists(go2w_scene_path):
    print(f"✓ Found Go2-W model at: {go2w_scene_path}")
    
    # Load the model
    model = mujoco.MjModel.from_xml_path(go2w_scene_path)
    data = mujoco.MjData(model)
    
    print(f"\n=== Go2-W Model Information ===")
    print(f"Number of joints: {model.njnt}")
    print(f"Number of actuators: {model.nu}")
    print(f"Number of bodies: {model.nbody}")
    print(f"Simulation timestep: {model.opt.timestep} seconds")
else:
    print(f"✗ Go2-W model not found at: {go2w_scene_path}")
    print("\nLet's check what's available:")
    !ls -la unitree_mujoco/unitree_robots/

In [ ]:
# Print joint names - Go2-W should have MORE joints than Go2 (wheels!)
print("\n=== Go2-W Joint Names ===")
print("(Note: Go2-W has wheel joints in addition to leg joints)\n")

for i in range(model.njnt):
    name = mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_JOINT, i)
    print(f"  Joint {i:2d}: {name}")

In [ ]:
# Print actuator names
print("\n=== Go2-W Actuators ===")
print("(These are the motors you can control)\n")

for i in range(model.nu):
    name = mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_ACTUATOR, i)
    print(f"  Actuator {i:2d}: {name}")

### Step 5: Render the Go2-W

In [ ]:
# Create renderer
renderer = mujoco.Renderer(model, height=480, width=640)

# Reset simulation
mujoco.mj_resetData(model, data)

# Step simulation forward
for _ in range(200):
    mujoco.mj_step(model, data)

# Render
renderer.update_scene(data)
image = renderer.render()

# Display
plt.figure(figsize=(12, 8))
plt.imshow(image)
plt.axis('off')
plt.title('Unitree Go2-W (Wheel-Leg Hybrid Robot)', fontsize=14)
plt.show()

print("\n✓ Go2-W loaded successfully!")
print("Notice the WHEELS at the feet - this is what makes it Go2-W!")

### Step 6: Simple Animation - Test the Wheels!

In [ ]:
# Reset simulation
mujoco.mj_resetData(model, data)

# Simulation parameters
duration = 3.0  # seconds
framerate = 30

frames = []
sim_time = 0

print("Simulating Go2-W movement...")

while sim_time < duration:
    # Apply simple control
    # Try to make small movements to see the robot
    t = sim_time
    for i in range(min(model.nu, 16)):  # Go2-W has 16 actuators
        data.ctrl[i] = 0.3 * np.sin(2 * np.pi * 0.5 * t + i * 0.3)
    
    # Step simulation
    mujoco.mj_step(model, data)
    sim_time += model.opt.timestep
    
    # Capture frames
    if len(frames) < sim_time * framerate:
        renderer.update_scene(data)
        frames.append(renderer.render().copy())

print(f"Captured {len(frames)} frames")

In [ ]:
# Create GIF animation (no ffmpeg needed!)
from PIL import Image

# Convert frames to PIL images
pil_frames = [Image.fromarray(frame) for frame in frames]

# Save as GIF
pil_frames[0].save(
    'go2w_animation.gif',
    save_all=True,
    append_images=pil_frames[1:],
    duration=50,  # milliseconds per frame
    loop=0
)

# Display in notebook
from IPython.display import Image as IPImage, display
display(IPImage(filename='go2w_animation.gif'))

GO2PY Code